In [0]:
%sql
SELECT * FROM db_bike.`02_silver`.employees

In [0]:
%sql
SELECT * FROM db_bike.`02_silver`.order_details

In [0]:
%sql
SELECT * FROM db_bike.`02_silver`.order_headers

In [0]:
%sql
SELECT * FROM db_bike.`02_silver`.products

In [0]:
%sql
SELECT * FROM db_bike.`01_bronze`.bronze_business_partner

In [0]:
%sql
SELECT * FROM db_bike.`02_silver`.business_partners

- Missing two business partners, LEFT JOIN tables in products

In [0]:
%sql
-- CREATE TEMPORARY VIEW sales_orders AS 
SELECT 
  o.*
  ,p.*
  ,bp.*
FROM db_bike.`02_silver`.order_details o
INNER JOIN db_bike.`02_silver`.products p ON p.product_id = o.product_id
LEFT JOIN db_bike.`02_silver`.business_partners bp ON bp.partner_id = p.partner_id

In [0]:
display(_sqldf)

In [0]:
df_prod_1 = _sqldf

In [0]:
%sql
SELECT DISTINCT product_id FROM db_bike.`02_silver`.order_details WHERE product_id IN (SELECT product_id FROM db_bike.`02_silver`.products)

In [0]:
df_prod_2 = _sqldf

In [0]:
compare = df_prod_1.join(df_prod_2, "product_id", "left").filter(df_prod_1.product_id == df_prod_2.product_id)
display(compare)

In [0]:
%sql
SELECT * FROM db_bike.02_silver.order_details

In [0]:
def fact_sales():
    from pyspark.sql import functions as F
    dim_business_partners = spark.read.table("db_bike.03_gold.dim_partners").alias("bp")
    dim_products = spark.read.table("db_bike.03_gold.dim_products").alias("p")
    silver_order_details = spark.read.table("db_bike.02_silver.order_details").alias("o")

    fact_df = (
            silver_order_details
                .join(dim_products, F.col("o.product_id") == F.col("p.product_id"), how="inner")
                .join(dim_business_partners, F.col("bp.partner_id") == F.col("p.partner_id"), how="left")
                .withColumn("sales_sk", F.xxhash64(F.col("o.sales_order_id"), F.col("o.sales_order_item")).cast("String"))
                .withColumn("sales_amount", F.col("o.net_amount") * F.col("o.quantity"))
                .withColumn("date", F.to_date(F.col("o.delivery_date").cast("String"), "yyyyMMdd"))
                .select(
                    F.col("sales_sk")
                    ,F.col("bp.partner_sk")
                    ,F.col("p.product_sk")
                    ,F.col("date")
                    ,F.col("sales_amount") 
                )
    )
    return fact_df
display(fact_sales())